In [12]:
from audiodiffusion.audio_encoder import AudioEncoder
import torchaudio
import os
from tqdm import tqdm
import random as r
# from sklearn import KMeans
from sklearn.cluster import KMeans

In [25]:
#https://huggingface.co/teticio/audio-encoder
audio_encoder = AudioEncoder.from_pretrained("teticio/audio-encoder")

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


In [26]:
path = "data/rendered_audio/rendered_audio/"
files = os.listdir(path)
r.shuffle(files)
fs = []
embeddings = []

for file in tqdm(files[:100]):
    if ".flac" in file:
        full_path = path + file
        fs.append(full_path)
        audio, _ = torchaudio.load(full_path)
        encoding = audio_encoder.encode([full_path])
        embeddings.append(encoding.squeeze(0).numpy())



100%|████████| 100/100 [00:05<00:00, 18.25it/s]


In [27]:
kmeanModel = KMeans(n_clusters=2).fit(encodings)

In [28]:
encodings[4]

array([  36.537807  ,   50.830666  ,  -54.12584   ,   92.75671   ,
        -27.833855  ,   14.983583  ,   -2.5084019 ,   18.42607   ,
        -26.458656  ,   -1.8970935 ,    3.8541515 ,   66.29794   ,
        -19.557291  ,   50.102016  ,   -6.5089293 ,   50.396187  ,
         49.01052   ,   28.227583  ,  -17.893633  ,  -37.738766  ,
        -13.8838825 ,   62.095383  ,    8.665952  ,   70.00148   ,
        -38.611176  ,   99.22986   ,  -67.8202    ,   47.719288  ,
         45.841824  ,   49.857403  ,   40.50237   ,    0.19004533,
         -4.06607   ,   16.752302  ,  -36.260296  ,   28.942316  ,
        121.35594   ,   -2.9885635 ,   -2.756867  ,  100.18478   ,
         -2.4316032 ,  -35.24202   ,   18.645142  ,  -19.301155  ,
          1.8843265 ,   27.007269  ,   46.14515   ,  -58.897156  ,
        104.47957   ,   10.433294  ,    7.826271  ,  -25.132862  ,
         24.50602   , -132.44623   ,  -16.947086  ,  -63.83725   ,
          7.4761343 ,  -17.814379  ,   42.437332  ,   31.57403